In [1]:
import sys
sys.path.append('../src_variable')

import network as nw
import road as rd
import junction as jn
import traffic_lights as tl
import torch
import openGLUtils as glutils

### Testing right of way for some small road network

First consider a road system with 3 incoming roads and 3 outgoing roads. Order the roads from 1 to 6 and suppose that roads 1, 4 and 5 are incoming roads and 2, 3 and 6 are outgoing roads. 

Roads 1 to 4 represent a two directional road that has the right of way. Roads 5 and 6 represent a one-way road that crosses the junction.

We need to define some priorities and crossing_connections to represent this junction.

In [2]:
priorities = [[1,0,1],
              [0,1,2],
              [2,2,3]]

crossing_connections = [[[], [], []],
                        [[], [], [(0,0)]],
                        [[(0,0), (1,1), (1,2)], [(1,1)], [(0,0), (1,1)]]]

### Scenario 1:

All flux from road 1 goes to road 2, all flux from road 4 goes to road 3, all of the flux from road 5 goes to road 6. If the right of way functions as expected, when there is a lot of traffic on the road with right of way, almost none of the flux from road 5 will be able to enter road 6. Check that this is indeed the case.

Suppose that roads 1 and 4 start with a high density, and that roads 2 and 3 start with low densities so that there will be a lot of traffic on the road with right of way.

In [3]:
# Creating the roads
L = 50
N = 5

road1 = rd.Road(2, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 0), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.8, inflow = 0.1, id = "1_fw")
road2 = rd.Road(2, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.2, inflow = 0.6, id = "2_fw")
road3 = rd.Road(2, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (-1, 0),
                initial = lambda x: torch.ones_like(x) * 0.2, inflow = 0.6, id = "3_bw")
road4 = rd.Road(2, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (1, 0), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.8, inflow = 0.1, id = "4_bw")
road5 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 1), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.05, id = "5")
road6 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (0, -1),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.3, id = "6")

roads = [road1, road2, road3, road4, road5, road6]
entering = [0, 3, 4]
leaving = [1, 2, 5]

In [4]:
# Creating the junction -> No traffic lights for now

distribution = [[1.0, 0.0, 0.0],
                [0.0, 1.0, 0.0],
                [0.0, 0.0, 1.0]]
junction = jn.Junction(roads, entering=entering, leaving=leaving, distribution=distribution, trafficlights=[], 
                       coupled_trafficlights=[], duty_to_gw=True, priorities=priorities,
                       crossing_connections=crossing_connections)

In [5]:
# Create the network and make simulation
T = 150
network = nw.RoadNetwork(roads, [junction], T)
densities, _, _, _ = network.solve_cons_law()

In [8]:
# Make animation: running it here makes the kernel die
# glutils.draw_timed_with_shift(network, densities, interval_seconds = 0.05)

#### Same network but with both crossing and right hand rule:

In [9]:
# Creating the roads
L = 50
N = 5

road1 = rd.Road(2, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 0), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.8, inflow = 0.1, id = "1_fw")
road2 = rd.Road(2, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.2, inflow = 0.6, id = "2_fw")
road3 = rd.Road(2, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (-1, 0),
                initial = lambda x: torch.ones_like(x) * 0.2, inflow = 0.6, id = "3_bw")
road4 = rd.Road(2, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (1, 0), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.8, inflow = 0.1, id = "4_bw")
road5 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 1), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.05, id = "5")
road6 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (0, -1),
                initial = lambda x: torch.ones_like(x) * 0.3, inflow = 0.3, id = "6")

roads = [road1, road2, road3, road4, road5, road6]
entering = [0, 3, 4]
leaving = [1, 2, 5]

In [10]:
distribution = [[1.0, 0.0, 0.0],
                [0.0, 1.0, 0.0],
                [0.4, 0.4, 0.2]]

junction = jn.Junction(roads, entering=entering, leaving=leaving, distribution=distribution, trafficlights=[], 
                       coupled_trafficlights=[], duty_to_gw=True, priorities=priorities,
                       crossing_connections=crossing_connections)

In [11]:
# Create the network and make simulation
T = 150
network = nw.RoadNetwork(roads, [junction], T)
densities, _, _, _ = network.solve_cons_law()

In [12]:
glutils.draw_timed_with_shift(network, densities, interval_seconds = 0.05)

: 